In [1]:
import pyspark
import os
import zipfile

In [11]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd

In [8]:
# zip_ref = zipfile.ZipFile("/home/zxr_nju/data/attributes.csv.zip", 'r')
# zip_ref.extractall("/home/zxr_nju/data/")
# zip_ref.close()

In [14]:
sc = pyspark.SparkContext.getOrCreate()

In [15]:
ss = SparkSession.builder.getOrCreate()

In [16]:
attributes = sc.textFile("/home/zxr_nju/data/attributes.csv")

In [17]:
attributes.take(3)

['app_id,session_id,attribute,attribute_value,user_id_hash',
 '4724682771660800,2201961907282901522,0,1,9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006',
 '4724682771660800,2201961907282901522,1,1,9943447915df3a45fd6720a026af905b6da6b56a37701b8b2629802e9a541006']

In [9]:
attributes.map(lambda x: x.split(",")[-1]).distinct().take(5)

['41f01b5e081ebb28756c50f308f39f3598fee1a424a502cee4c7cfe36e5cfe3d',
 '43db4e4365c088cf2e148f7c650ea991f6e387006aaab2c94fe793c43f878d0a',
 'e8a532bc6e8cb35bfcb5af1c0ff5760e6498b41cbcae274fb291baf54142f0b3',
 '34923775eb87e845af4636430f78cb322f65c0fd4acad92176d58647bff05df6',
 '8e55a8232de00c2cd548081726f82821667e4fe76ee2b80e054fdf16a031851c']

In [18]:
attr = attributes.map(lambda x: [x.split(',')[0],x.split(',')[1],x.split(',')[2],\
                                 ','.join(x.split(',')[3:-1]).replace("\"","").replace("\\",""),x.split(',')[-1]])

In [19]:
header = attr.first()

In [20]:
header

['app_id', 'session_id', 'attribute', 'attribute_value', 'user_id_hash']

In [21]:
fields = [StructField(field_name, StringType(), True) for field_name in header]

In [22]:
schema = StructType(fields)

In [23]:
attr = attr.filter(lambda x: 'app_id' not in x)

In [24]:
attr_df = ss.createDataFrame(attr, schema =schema)

In [25]:
attr_df.show(30)

+----------------+-------------------+---------+--------------------+--------------------+
|          app_id|         session_id|attribute|     attribute_value|        user_id_hash|
+----------------+-------------------+---------+--------------------+--------------------+
|4724682771660800|2201961907282901522|        0|                   1|9943447915df3a45f...|
|4724682771660800|2201961907282901522|        1|                   1|9943447915df3a45f...|
|4724682771660800|2201961907282901522|        3|                   0|9943447915df3a45f...|
|4724682771660800|2201961907282901522|        4|                0.00|9943447915df3a45f...|
|4724682771660800|2201961907282901522|        5|romance_the_royal...|9943447915df3a45f...|
|4724682771660800|2201961907282901522|        6|                  29|9943447915df3a45f...|
|4724682771660800|2201961907282901522|        7|romance_the_royal...|9943447915df3a45f...|
|4724682771660800|2201961907282901522|        8|                   2|9943447915df3a45f...|

In [21]:
sc.stop()